In [9]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from tensorflow.keras.models import load_model
from scipy.spatial.distance import euclidean
import simpleaudio as sa
import threading

# Memuat model LSTM yang telah dilatih
model = load_model('lstm_model.h5')

# Inisialisasi detektor wajah dlib dan prediktor landmark wajah
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Mendefinisikan konstanta 
SEQ_LENGTH = 5  # Panjang urutan yang diperlukan oleh model LSTM
MICROSLEEP = 10  # Jumlah frame berturut-turut untuk mendeteksi kantuk

# Inisialisasi counter frame dan status kantuk
COUNTER = 0
DROWSY = False
is_playing = False

# Inisialisasi list untuk menyimpan urutan fitur
seq = []

# Fungsi untuk menghitung Eye Aspect Ratio (EAR) menggunakan jarak Euclidean
def calculate_ear(eye):
    A = euclidean(eye[1], eye[5])  # Jarak vertikal antara titik 1 dan 5
    B = euclidean(eye[2], eye[4])  # Jarak vertikal antara titik 2 dan 4
    C = euclidean(eye[0], eye[3])  # Jarak horizontal antara titik 0 dan 3
    ear = (A + B) / (2.0 * C)  # Menghitung EAR
    return ear

# Fungsi untuk menghitung Mouth Aspect Ratio (MAR) menggunakan jarak Euclidean
def calculate_mar(mouth):
    A = euclidean(mouth[2], mouth[10])  # Jarak vertikal antara titik 2 dan 10
    B = euclidean(mouth[4], mouth[8])  # Jarak vertikal antara titik 4 dan 8
    C = euclidean(mouth[0], mouth[6])  # Jarak horizontal antara titik 0 dan 6
    mar = (A + B) / (2.0 * C)  # Menghitung MAR
    return mar

# Fungsi untuk menghitung Mouth Over Eye (MOE) menggunakan jarak Euclidean
def calculate_moe(mar, ear):
    return mar / ear  # Menghitung MOE sebagai rasio antara MAR dan EAR

# Fungsi untuk memperbarui urutan dengan data baru dan membuat prediksi
def update_sequence(seq, new_data):
    seq.append(new_data)  # Menambahkan data baru ke urutan
    if len(seq) > SEQ_LENGTH:
        seq.pop(0)  # Jika panjang urutan melebihi SEQ_LENGTH, hapus elemen pertama
    return seq  # Mengembalikan urutan sebagai list

# Fungsi untuk mereset keadaan
def reset_state():
    global COUNTER, DROWSY, seq
    COUNTER = 0
    DROWSY = False
    seq = []

# Fungsi untuk memainkan suara peringatan di thread terpisah
def play_sound():
    global is_playing
    if not is_playing:
        is_playing = True
        wave_obj = sa.WaveObject.from_wave_file("alarm.wav")
        play_obj = wave_obj.play()
        play_obj.wait_done()
        is_playing = False
        reset_state()  # Reset state setelah alarm selesai diputar



# Inisialisasi stream video
cap = cv2.VideoCapture(0)

# Tentukan ukuran jendela tampilan yang diinginkan
window_name = "Frame"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 1200, 1200)  # Ukuran jendela 1280x720

while True:
    ret, frame = cap.read()  # Membaca frame dari kamera
    if not ret:
        break  # Jika tidak ada frame yang dibaca, keluar dari loop

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Mengkonversi frame ke grayscale
    rects = detector(gray, 0)  # Menggunakan detektor wajah dlib untuk mendeteksi wajah dalam frame grayscale

    for rect in rects:  # Iterasi melalui setiap deteksi wajah
        shape = predictor(gray, rect)  # Mendapatkan landmark wajah
        shape = face_utils.shape_to_np(shape)  # Mengonversi landmark wajah ke array NumPy

        # Debugging output untuk memeriksa shape
        print("Shape size:", shape.shape)

        # Ekstraksi koordinat mata kiri dan kanan serta mulut
        if shape.shape[0] >= 68:  # Memastikan bahwa shape memiliki ukuran yang diharapkan
            left_eye_start, left_eye_end = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
            right_eye_start, right_eye_end = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
            mouth_start, mouth_end = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

            leftEye = shape[left_eye_start:left_eye_end]
            rightEye = shape[right_eye_start:right_eye_end]
            mouth = shape[mouth_start:mouth_end]

            leftEAR = calculate_ear(leftEye)
            rightEAR = calculate_ear(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            
            

            mar = calculate_mar(mouth)
            moe = calculate_moe(mar, ear)

            # Menyiapkan vektor fitur
            feature_vector = [ear, mar, moe]

            # Memperbarui urutan dengan vektor fitur baru
            seq = update_sequence(seq, feature_vector)

            # Membuat prediksi jika kita memiliki cukup data dalam urutan
            if len(seq) == SEQ_LENGTH:
                input_seq = np.array(seq).reshape(1, SEQ_LENGTH, 3)  # Mengonversi urutan menjadi array NumPy untuk prediksi
                prediction = model.predict(input_seq)
                drowsiness_prob = prediction[0][0]
                is_drowsy = drowsiness_prob > 0.5

                if is_drowsy:  # Jika seseorang mengantuk
                    DROWSY = True
                    COUNTER += 1
                    if COUNTER >= MICROSLEEP:  # Jika counter melebihi threshold, tampilkan peringatan
                        cv2.putText(frame, "AWAS NGANTUK!", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        
                        if not is_playing:
                            threading.Thread(target=play_sound).start()  # Mainkan suara peringatan di thread terpisah

                else:
                    DROWSY = False
                    cv2.putText(frame, "Siaga", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    if COUNTER > 0:
                        COUNTER -= 1  # Kurangi counter jika tidak mengantuk
                        
                    

            # Menampilkan nilai EAR, MAR, dan MOE pada frame
            cv2.putText(frame, f'EAR: {ear:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'MAR: {mar:.2f}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'MOE: {moe:.2f}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'COUNTER: {COUNTER}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            

            # Menggambar kontur mata kiri dan kanan serta mulut
            for (x, y) in leftEye:
                cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
            for (x, y) in rightEye:
                cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
            for (x, y) in mouth:
                cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
    
    cv2.imshow(window_name, frame)  # Menampilkan frame dengan OpenCV
    key = cv2.waitKey(1) & 0xFF  # Menunggu input tombol

    if key == ord("q"):  # Jika tombol 'q' ditekan, keluar dari loop
        break

cap.release()  # Melepaskan sumber daya video capture
cv2.destroyAllWindows()  # Menutup semua jendela OpenCV


Shape size: (68, 2)
Shape size: (68, 2)
Shape size: (68, 2)
Shape size: (68, 2)
Shape size: (68, 2)
1/1 [==============================] - 1s 891ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 21ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 21ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 21ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 20ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 22ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 21ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 21ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 23ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 21ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 21ms/step
Shape size: (68, 2)
1/1 [==============================] - 0s 23ms/step
Shape size: (68, 2)
1/1 [==============================